In [1]:
import os, datetime
import numpy as np
import pandas as pd

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression, Ridge, Lasso

In [2]:
# train 데이터는 '2019-01-01' ~ '2019-08-31'
# test 데이터는  '2019-09-01' ~ '2019-12-31'

data = pd.read_csv('data.csv', parse_dates=['datetime'])
train = data.set_index('datetime').loc['2019-01-01':'2019-08-31'].reset_index()
test = data.set_index('datetime').loc['2019-09-01':'2019-12-31'].reset_index()
solution = pd.read_csv('data_solution.csv', parse_dates=['datetime'])

building = pd.read_csv('building_metadata.csv')
weather = pd.read_csv('weather.csv', parse_dates=['datetime'])
weather_train = weather.set_index('datetime').loc['2019-01-01':'2019-08-31'].reset_index()
weather_test = weather.set_index('datetime').loc['2019-09-01':'2019-12-31'].reset_index()
weather_valid = pd.read_csv('weather_valid.csv', parse_dates=['datetime'])

temperature = pd.read_csv('temperature.csv', parse_dates=['datetime'])

# 데이터 체크
print('data size:', data.shape)
print('train size:', train.shape)
print('test size:', test.shape)
print('solution size:', solution.shape)

print('weather size:', weather.shape)
print('weather train size:', weather_train.shape)
print('weather test size:', weather_test.shape)
print(weather_train.shape[0] + weather_test.shape[0])
print(365 * 24 * 13)

print('weather valid size:', weather_valid.shape)
print(28 * 24)

print('temperature size:', temperature.shape)
print(365 * 24)

print('building_metadata size:', building.shape)

data size: (300404, 3)
train size: (198879, 3)
test size: (101525, 3)
solution size: (101525, 3)
weather size: (113880, 8)
weather train size: (75816, 8)
weather test size: (38064, 8)
113880
113880
weather valid size: (672, 3)
672
temperature size: (8760, 52)
8760
building_metadata size: (1449, 6)


In [3]:
# Problem 1

# Nan인 데이터 값들을 채워넣는 과정이다.

# Restore를 수행할 column들
columns_to_restore = ['temperature','dew_point','cloud','rain_hourly','air_pressure','wind_direction']

# train과 test 날씨 데이터를 결합한다.
weather_df = pd.concat([weather_train, weather_test])

# region_id와 datetime으로 인덱스를 설정하여 탐색 속도를 높인다.
weather_df = weather_df.set_index(['region_id', 'datetime']).sort_index()

# interpolation을 이용하여 비어있는 값들을 채워야 한다.
# 그렇기 위해서는 먼저 데이터를 time series 데이터로 추출해야 한다.
# 각각의 region 마다 독립적인 time series 데이터가 있기 때문에 time series 별로 따로 추출해야 한다.
for column in columns_to_restore:
    for region_id in range(13):
        s = weather_df.loc[region_id, column].copy()
        
        # interpolation으로 값을 채울때 limit_direction을 both로 선택하여 모든 값들을 채울 수 있도록 한다.
        # cloud 칼럼의 경우 0, 2, 4, 6, 8로 이루어진 값이기 때문에 
        # 해당 값들로 정수화 하는 과정이 추가로 필요하다.
        if column == 'cloud':
            interpolation = (s/2).interpolate(limit_direction='both')
            interpolation[interpolation.notnull()] = interpolation[interpolation.notnull()].astype(np.int8)*2
        else:
            interpolation = s.interpolate(limit_direction='both')
        weather_df.loc[region_id, column].iloc[:] = interpolation
        
# 주어진 valid 데이터(weather_valid.csv)를 이용하여 interpolate의 정확도를 테스트한다.
# valid 데이터는 region_id = 9 의 2월 데이터가 빠짐없이 들어있다.
# 이를 이용하여 interpolate된 값과 correlation을 계산한다.
column = columns_to_restore[0]
region_id = 9
st  = '2019-02-01 00:00:00'
end = '2019-02-28 23:59:59'
index_slice = slice(st, end)

weather_valid_df = weather_valid.copy()
weather_valid_df = weather_valid_df.set_index(['region_id', 'datetime'], drop=False).sort_index()

# RMSE 값을 계산하여 출력한다.
rmse = ((weather_df.loc[region_id, column] - weather_valid_df.loc[region_id, column]) ** 2).mean() ** .5
print('Answer:', round(rmse,3))

Answer: 0.416


In [4]:
# Problem 2

# 현재 주어진 데이터의 region은 번호로 주어져 있고 실제 도시 이름을 모른다.
# 이를 다른 데이터를 이용하여 실제 도시 이름을 유추해본다.
# 이때 다른 데이터와 비교할 metric으로 dynamic time warping (DTW)를 사용한다.
# temperature.csv에서는 여러 도시별 시간별 기온이 들어있다.
# weather 데이터와 temperature 추가 데이터의 datetime은 같은 표준시에 맞춰져 있기 떄문에 
# 따로 datetime를 처리할 필요는 없다.

weather_df = weather_df.reset_index()

# weather 데이터에서 temperature 데이터를 추출한다.
df_temperature_pivot = weather_df.reset_index().pivot_table(index='datetime', columns='region_id', values='temperature')
df_temperature_pivot = df_temperature_pivot.loc[:,[0,1]]

# 추가 데이터의 temperature 데이터를 가져와서 weather 데이터와 datetime을 이용하여 merge 한다.
# 그러기 위해서 양쪽 데이터의 index를 datetime으로 설정해 두어야 한다.
# merge 할때 key join 방법으로 inner를 선택한다.
temperature_df = temperature.copy()
temperature_df.set_index('datetime', inplace=True)
temperature_df = temperature_df.merge(df_temperature_pivot, left_index=True, right_index=True, how='inner')

print(temperature_df.shape[0], temperature_df.shape[1]+1) # datetime column Count 추가

# merge된 데이터에서 weather 데이터와 temperature 추가 데이터를 분리한다.
col_regions = [i for i in range(2)]
col_cities = [col for col in temperature_df.columns if col not in col_regions]
temperature_regions = [temperature_df[col].copy() for col in temperature_df.columns if col in col_regions]
temperature_cities = [temperature_df[col].copy() for col in temperature_df.columns if col in col_cities]

# RMSE를 계산한다.
def calc_rmse(a, b):
    return (((a - b)**2).abs()).mean()**(1/2)

rmse_table = []
for i, temperature_city in enumerate(temperature_cities):
    rmse_row = []
    for j, temperature_region in enumerate(temperature_regions):
        rmse = calc_rmse(temperature_region, temperature_city)
        rmse_row.append(rmse)
        
    rmse_table.append(rmse_row)
    
df_rmse = np.array(rmse_table)

# RMSE가 가장 작은 index및 그 값을 찾아서 dataframe으로 만든다.
df_findCity_indicies = df_rmse.argmin(axis=0)
df_findCity_rmse = df_rmse.min(axis=0)
df_findCity = pd.DataFrame.from_dict({
    #'dtw': df_findCity_rmse,
    'city': np.array(col_cities)[df_findCity_indicies],
    'region': col_regions,
})

# matching된 city들을 출력한다.
print('Answer:', df_findCity['city'][0]+', '+df_findCity['city'][1])

8760 54
Answer: minneapolis, austin


In [5]:
# Problem 3

# train 데이터와 building 데이터와 weather 데이터를 merge 한다.
# merge 할때 key joint 방법은 'left'를 사용한다.
# join 할때 사용할 key 값들로 region_id와 date_time를 사용한다.
train_df = train.copy()
train_df = train_df.merge(building, on='user_id', how='left')
train_df_merged = train_df.merge(weather_df, on=['region_id', 'datetime'], how='left')
print(train_df_merged.shape)

df = train_df_merged.copy()
df['hr'] = df['datetime'].dt.hour
df = df.groupby(['region_id', 'hr']).mean()

# column은 time에 대해 변하는 변수들만 사용한다. user_id나 area 등 시간에 독립적인 값들은 제외한다.
corrs = []
df = df.reset_index()
columns = ['generate', 'temperature', 'cloud', 'dew_point', 'rain_hourly', 'air_pressure', 'wind_direction']

total_corr = np.zeros([len(columns), len(columns)])
count = np.zeros([len(columns), len(columns)])

# time series는 region 별로 독립적이기 때문에 region_id별로 추출해서 계산해야 한다.
# 각 region 별로 correation 값을 계산후 이를 마지막에 평균으로 계산한다. 
for region_id in df['region_id'].unique():
    # region_id (2, 4, 7, 12) 에는 NaN 값이 있기 때문에 해당 데이터들은 제외한다.
    if region_id in [2, 4, 7, 12]:
        continue

    series = df[df['region_id'] == region_id]
    series = series[columns]

    # correation 값을 계산한다. self-correlation을 계산하면 모든 column 조합의 correlation이 나온다.
    corr = np.array(series.corr('pearson'))

    # correation 값을 더해준다.
    total_corr += corr
    count += 1

# total_corr를 count로 나눠주면 평균 correlation 값이 나온다.
mean_corr = total_corr / count
max_index = np.argmax(mean_corr[0][1:])
print('Answer:',columns[max_index+1],round(mean_corr[0][max_index+1],3))

(198879, 14)
Answer: air_pressure 0.255


In [6]:
# Problem 4

# weather 데이터가 local 시간에 맞춰져 있지 않고 표준시이기 때문에 데이터가 shift 되어 있다.
# 이에 반해 train 데이터는 local 시간에 맞춰져 있다.
# 학습 성능 향상을 위해 weather 데이터를 local 시간에 맞추는 작업을 한다.

weather_df = weather_df.reset_index(drop=True)
weather_key = ['region_id', 'datetime']
temp_series = weather_df[weather_key + ['temperature']].drop_duplicates(subset=weather_key).sort_values(by=weather_key).copy()

# 각 region별, 시간별 데이터를 평균낸다.
df_temp = temp_series.groupby(['region_id', temp_series.datetime.dt.hour])['temperature'].mean().unstack(level=1)

# 기온의 peak가 14시가 되도록 맞춘다. (오후 2시가 가장 덥다는 가정이다.)
# peak 값이 14에서 얼마나 떨어져 있는지 offset을 계산한다.
region_ids_offsets = pd.Series(df_temp.values.argmax(axis=1) - 14)
region_ids_offsets.index.name = 'region_id'

# 계산된 값을 weather 데이터의 datetime에서 빼준다. 이때 offset 값을 timedelta hour로 변환해야한다.
weather_df_aligned = weather_df.copy()
weather_df_aligned['offset'] = weather_df_aligned.region_id.map(region_ids_offsets)
weather_df_aligned['datetime'] = (weather_df_aligned.datetime - pd.to_timedelta(weather_df_aligned.offset, unit='H'))

train_df_merged_aligned = train_df.merge(weather_df_aligned, on=['region_id', 'datetime'], how='left')                    
train_df_merged_aligned = train_df.merge(weather_df_aligned, on=['region_id', 'datetime'], how='left')



train_df = train_df_merged_aligned.copy()

not_null_list = pd.notnull(train_df['construct_year'])

df = train_df.copy()[not_null_list]
print(df.shape)

# error metric을 Root Mean Squared Logarithmic Error 을 사용한다.
df['generate'] = np.log1p(df['generate'])#.astype(np.float32)

# datetime을 그대로 사용하지 않고 분리해서 사용한다.
# 전체 시간에 대해 regression하는 것보다 월별, 일별, 시간별로 regression 하는게 더 정확하다.
df['hour'] = np.uint8(df['datetime'].dt.hour)
df['day'] = np.uint8(df['datetime'].dt.day)
df['month'] = np.uint8(df['datetime'].dt.month)

# datetime 칼럼을 제거한다.
for col in df.columns:
    if col in ['datetime']:
        del df[col]
    
df_train = df

# feature들을 따로 빼준다.
# 먼저 예측할 값인 generate를 제외하고 추가로 categorical feature들인 'user_id', 'usage', 'region_id'를 제거한다.
# 위에서 offset 칼럼이 생겼기 때문에 이 또한 제거한다. 
# 최종적으로 사용되는 feature들은 아래와 같다.
# ['area', 'construct_year', 'num_floors', 'temperature', 'cloud', 'dew_point', 'rain_hourly', 'air_pressure',
# 'wind_direction', 'hour', 'day', 'month']
target = 'generate'
categorical_features = ['user_id', 'usage', 'region_id']
feature_cols = [col for col in df_train.columns if col not in [target, 'offset'] + categorical_features]
# print('Used features:', feature_cols)

features = df_train[feature_cols]

# Nan인 값들은 0으로 채워준다.
features[features.isnull()] = 0
    
label = df_train[target]

train_features = features.copy()
train_label = label.copy()

# Linear Regression
clf = LinearRegression(fit_intercept=True, normalize=False)
clf.fit(train_features, train_label)

# train 데이터 에러값을 계산하고 위해 train 데이터의 feature들을 이용하여 predict 한다.
predicted = clf.predict(train_features)
expected = label

pd.DataFrame.from_dict({
    'features': feature_cols,
    'std': train_features.std(),
    'coef': clf.coef_,
    'importance': np.abs(clf.coef_) * train_features.std()
})

# RMSE 출력
print('Answer:',round(np.sqrt(np.mean((predicted - expected) ** 2)),3) )

(91140, 15)


c:\users\giwoong.bae\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\giwoong.bae\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:3515: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)


Answer: 1.661


In [7]:
# Problem 5

train_df = train_df_merged_aligned.copy()

df = train_df.copy()[not_null_list]

df['generate'] = np.log1p(df['generate'])#.astype(np.float32)

df['hour'] = np.uint8(df['datetime'].dt.hour)
df['day'] = np.uint8(df['datetime'].dt.day)
df['month'] = np.uint8(df['datetime'].dt.month)

for col in df.columns:
    if col in ['datetime', 'row_id']:
        del df[col]
    
# usage는 categorical feature이기 때문에 regression에 사용할 수 없다.
# usage를 feature로 사용하기 위해 usage를 continuous 값으로 변환한다.
# 각각의 usage 별로 generate 값의 평균을 내고 해당 평균을 usage 값으로 사용한다.
# 이 과정을 log 도메인에서 진행해야 한다. (log1p 함수 이후에 수행해야함)
usage_scale = df.groupby('usage', as_index=False)['generate'].mean()
usage_scale.columns = ['usage', 'usage_numeric']
df = pd.merge(df, usage_scale, how='left', on='usage')

# area와 num_floors feature들을 log 도메인으로 변환시켜준다.
df['area'] = np.log1p(df['area'])
df['num_floors'] = np.log1p(df['num_floors'])
    
df_train = df

target = 'generate'
categorical_features = ['user_id', 'usage', 'region_id']
feature_cols = [col for col in df_train.columns if col not in [target, 'offset'] + categorical_features]

features = df_train[feature_cols]

features[features.isnull()] = 0
    
label = df_train[target]

train_features = features.copy()
train_label = label.copy()

clf = Ridge(alpha=0.001, max_iter=1000, tol=1e-3, random_state=1234)
clf.fit(train_features, train_label)

predicted = clf.predict(train_features)
expected = label

pd.DataFrame.from_dict({
    'features': feature_cols,
    'std': train_features.std(),
    'coef': clf.coef_,
    'importance': np.abs(clf.coef_) * train_features.std()
})

print('Answer:',round(np.sqrt(np.mean((predicted - expected) ** 2)),3))

Answer: 1.523


c:\users\giwoong.bae\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
c:\users\giwoong.bae\appdata\local\programs\python\python37\lib\site-packages\pandas\core\frame.py:3515: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._where(-key, value, inplace=True)
